# Generate the event and article knowledge graph

In [6]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import ast
from resources import *
import re

rnews = Namespace("http://iptc.org/std/rNews/2011-10-07#")
nif = Namespace("http://persistence.uni-leipzig.org/nlp2rdf/ontologies/nif-core#")
faro = Namespace("https://purl.org/faro/")
sem = Namespace("http://semanticweb.cs.vu.nl/2009/11/sem/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
rnews = Namespace("http://iptc.org/std/rNews/2011-10-07#")
schema = Namespace("http://schema.org/")

faro_classes = {'cause': faro.causes, 'enable': faro.enables, 'intend': faro.intends_to_cause, 'prevent': faro.prevents} #dict of faro definitions
sem_props = {'http://www.wikidata.org/prop/direct/P710': sem.hasActor,
             'http://www.wikidata.org/prop/direct/P664': sem.hasActor,
             'http://www.wikidata.org/prop/direct/P112': sem.hasActor,
             'http://www.wikidata.org/prop/direct/P17': sem.hasPlace,
             'http://www.wikidata.org/prop/direct/P276': sem.hasPlace,
             'http://www.wikidata.org/prop/direct/P625': sem.hasPlace,
             'http://www.wikidata.org/prop/direct/P131': sem.hasPlace,
             'http://www.wikidata.org/prop/direct/P30': sem.hasPlace,
             'http://www.wikidata.org/prop/direct/P585': sem.hasTime,
             'http://www.wikidata.org/prop/direct/P580': sem.hasBeginTimeStamp,
             'http://www.wikidata.org/prop/direct/P582': sem.hasEndTimeStamp,
             'http://www.wikidata.org/prop/direct/P571': sem.hasTime,
             'http://www.wikidata.org/prop/direct/P576': sem.hasTime,
             'http://www.wikidata.org/prop/direct/P577': sem.hasTimeStamp,
             'http://www.w3.org/2000/01/rdf-schema#label': 'what'}

sem_classes = {sem.hasActor: sem.Actor,
               sem.hasPlace: sem.Place,
               sem.hasTime: sem.Time,
               sem.hasBeginTimeStamp: sem.Time,
               sem.hasEndTimeStamp: sem.Time,
               sem.hasTimeStamp: sem.Time,
               'what': sem.Event}

In [3]:
def clean_text(text):

    # Strip the last part of the text
    index_of_last_occurence = text.rfind('</p><p>')
    if index_of_last_occurence != -1:
        text = text[:index_of_last_occurence]

    text = re.sub(r"<.*?>", " ", text) # Strip all the special characters in the text

    text = text.strip() #Remove the whitespace at the beginning, due to deletion

    return text

In [7]:
data = pd.read_csv('Data/ASRAEL_data_full.csv')
data['Text'] = data['Text'].apply(clean_text)

In [8]:
#This converts the data into the right format, by removing uneccessary tokens in text and converting the wikidata link to text
graph = Graph()

event_mapping = {} #here the wikidata event urls and their names are saved
failed_events= [] #Events that can't be found e.g. owl:sameAs need to be removed

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

for event in data['Event'].unique().tolist():
    event_ = f"wd:{event.split('/')[-1]}"

    sparql.setQuery("""
    SELECT (?p as ?wiki_prop) (?o as ?result)
    WHERE {{{

        %s ?p ?temp.
      ?temp rdfs:label ?o.
      FILTER (lang(?o) = 'en') }
      }

      UNION

      {
       SELECT *
       WHERE{
        %s ?p ?o.
         FILTER(lang(?o) = 'en' || lang(?o)='') }}
     } """ % (event_, event_))

    try:
        result = sparql.queryAndConvert()
        #event_name = ret['results']['bindings'][0]['item']['value']

    except:
        print(f"Something went wrong when converting event: {event}")

    try:
        event_data = pd.json_normalize(result["results"]["bindings"])[['wiki_prop.value', 'result.value', 'result.datatype']]
        event_data = event_data.rename(columns={"wiki_prop.value": "property", "result.value": "value", "result.datatype": "datatype"})
        event_data = event_data.loc[event_data['property'].isin(sem_props.keys())].reset_index(drop=True) #Only keep the 4W attributes
        event_data['property'] = event_data['property'].replace(sem_props)
        event_name = event_data.loc[event_data['property'] == 'what']['value'].values[0] #This needs to be saved to map the wikidata urls to events
        event_data = event_data[event_data.property != 'what'] #This row needs to be deleted for the loop
        event_mapping[event] = event_name
    except:
        print(f"Error when searching for event: {event}")
        failed_events.append(event)
        continue

    #event_uri = node_creation('', event_name, base_add='/event') #Generate the URI for the event
    graph.add((URIRef(event), RDF.type, sem.Event)) #Create the event
    graph.add((URIRef(event), RDF.value, Literal(event_name)))

    for index, row in event_data.iterrows():
        uri = node_creation('', row['value'], base_add='') #Generate the URI for the property
        graph.add((uri, RDF.type, sem_classes[row['property']])) #Create the node for the property, and lookup its class
        if pd.isna(row['datatype']) == False: #It has a declared datatype
            graph.add((uri, RDF.value, Literal(row['value'], datatype=row['datatype'])))
        else:
            graph.add((uri, RDF.value, Literal(row['value']))) #Add the value of the relation to the graph
        graph.add((URIRef(event), row['property'], uri)) #Connect the event to the property

graph.serialize('Data/examples/Event_graph_all.ttl', format='turtle')

data= data[~data['Event'].isin(failed_events)] #remove the rows for which the event was not found
#data['Event'] = data['Event'].map(event_mapping)
#Check if this still is oke, it adds a list of events to the column event
data = data.groupby(['URI','Identifier','Location', 'Time', 'Text']).agg({'Event': lambda x: list(x)}).reset_index(drop=False)
data.to_csv('Data/ASRAEL_data_full_converted.csv')


Error when searching for event: http://www.wikidata.org/entity/Q100919128
Error when searching for event: http://www.wikidata.org/entity/Q113945893
Error when searching for event: http://www.wikidata.org/entity/Q105597606


In [13]:
from nltk import tokenize
data = pd.read_csv('Data/ASRAEL_data_full_converted.csv')
data2 = pd.read_csv('Data/final_data_with_predictions.csv')
events = []
for index, row in data.iterrows():
    for sentence in tokenize.sent_tokenize(row['Text']):
        events.append(row['Event'])


data2['Event'] = events
data2.to_csv('Data/final_data_with_predictions.csv')

In [2]:
from tqdm import tqdm
from nltk import tokenize

graph = Graph()
graph.parse('Data/examples/Event_graph_all.ttl')
#data = pd.read_csv('Data/ASRAEL_data_full_converted.csv')
data = pd.read_csv('Data/final_data_with_predictions_new.csv')

data_with_predictions = True

for index, row in tqdm(data.iterrows(), total=data.shape[0]):

    if not row.isnull().values.any(): #If nan is present skip the row

        graph.add((URIRef(row['URI']), RDF.type, rnews.Article)) #Add the URI as article

        identifier_uri = node_creation('', row['Identifier'], base_add='/identifier')
        graph.add((URIRef(row['URI']), rnews.identifier, URIRef(identifier_uri))) #Link the PublicID to the article
        graph.add((URIRef(identifier_uri), RDF.value, Literal(row['Identifier'])))

        location_uri = node_creation('', row['Location'], base_add='')
        graph.add((URIRef(row['URI']), schema.contentLocation, URIRef(location_uri)))
        graph.add((URIRef(location_uri), RDF.value, Literal(row['Location'])))

        time_uri = node_creation('', row['Time'], base_add='')
        graph.add((URIRef(row['URI']), schema.contentReferenceTime, URIRef(time_uri)))
        graph.add((URIRef(time_uri), RDF.value, Literal(row['Time'])))

        if data_with_predictions == False: #Make the predictions

            from rebel_finetuning_faro import make_predictions

            sentences, predictions = make_predictions(tokenize.sent_tokenize(row['Text']), 'rebel_finetuned.pth')
            for sentence, prediction in zip(sentences, predictions):

                if prediction[1] in faro_classes:
                    sentence_uri = node_creation('', sentence, base_add='/sentence') #Generate the URI for the sentence
                    graph.add((URIRef(row['URI']), nif.sentence, sentence_uri)) #Link the article to the sentence
                    graph.add((sentence_uri, RDF.type, nif.Sentence)) #Make the sentence URI of class 'Sentence'
                    graph.add((sentence_uri, RDF.value, Literal(sentence))) #Set the value of the URI equal to the sentence

                    subject_uri = node_creation('', prediction[0] + str(sentence_uri), base_add='/subject') #Generate the URI for the subject, for now add the uri of sentence to make it unique
                    graph.add((sentence_uri, faro.Relata, subject_uri)) #Add the subject to the sentence
                    graph.add((subject_uri, RDF.value, Literal(prediction[0]))) #Set the value of the subject URI equal to the subject

                    object_uri = node_creation('', prediction[2] + str(sentence_uri), base_add='/object') #Generate the URI for the object, for now add the uri of sentence to make it unique
                    graph.add((sentence_uri, faro.Relata, object_uri)) #Add the object to the sentence
                    graph.add((object_uri, RDF.value, Literal(prediction[2]))) #Set the value of the subject URI equal to the object

                    graph.add((subject_uri, faro_classes[prediction[1]], object_uri)) #Add relation betwee NERs
                else:
                    continue
        else: # The data already contains the predictions
            sentence = row['Sentence']
            prediction = (row['Subject'], row['Relation'], row['Object'])

            if prediction[1] in faro_classes:
                sentence_uri = node_creation('', sentence, base_add='/sentence') #Generate the URI for the sentence
                graph.add((URIRef(row['URI']), nif.sentence, sentence_uri)) #Link the article to the sentence
                graph.add((sentence_uri, RDF.type, nif.Sentence)) #Make the sentence URI of class 'Sentence'
                graph.add((sentence_uri, RDF.value, Literal(sentence))) #Set the value of the URI equal to the sentence

                subject_uri = node_creation('', prediction[0] + str(sentence_uri), base_add='/subject') #Generate the URI for the subject, for now add the uri of sentence to make it unique
                graph.add((sentence_uri, faro.Relata, subject_uri)) #Add the subject to the sentence
                graph.add((subject_uri, RDF.value, Literal(prediction[0]))) #Set the value of the subject URI equal to the subject

                object_uri = node_creation('', prediction[2] + str(sentence_uri), base_add='/object') #Generate the URI for the object, for now add the uri of sentence to make it unique
                graph.add((sentence_uri, faro.Relata, object_uri)) #Add the object to the sentence
                graph.add((object_uri, RDF.value, Literal(prediction[2]))) #Set the value of the subject URI equal to the object

                graph.add((subject_uri, faro_classes[prediction[1]], object_uri)) #Add relation betwee NERs
            else:
                continue

        for event in ast.literal_eval(row['Event']): #Link the article to the corresponding event
            #event_uri = node_creation('', event_name, base_add='/event')
            graph.add((URIRef(row['URI']), schema.about, URIRef(event)))

graph.serialize('Data/examples/event_article_graph_complete_new.ttl', format='turtle')

100%|██████████| 19578/19578 [00:16<00:00, 1198.91it/s]


<Graph identifier=Nddb162272a924e7aaa69ed347461455e (<class 'rdflib.graph.Graph'>)>

In [ ]:
import networkx as nx
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph

nx_graph = rdflib_to_networkx_multidigraph(graph)
pos = nx.spring_layout(nx_graph, scale=2)

edge_labels = nx.get_edge_attributes(nx_graph, 'r')
nx.draw_networkx_edge_labels(nx_graph, pos, edge_labels=edge_labels)
nx.draw(nx_graph, with_labels=True)

In [2]:
# Perform predictions again since the sentence number is needed for event resolution
import pandas as pd
from rebel_finetuning_faro import make_predictions
from nltk import tokenize
from tqdm import tqdm

data = pd.read_csv('Data/ASRAEL_data_full_converted.csv')
total_uri = []
total_identifier = []
total_location = []
total_time = []
total_event = []
total_sentence_num = []
total_sentences = []
total_subject = []
total_relation = []
total_object = []

for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    sentences, predictions = make_predictions(tokenize.sent_tokenize(row['Text']), 'rebel_finetuned.pth')
    for i, (sentence, prediction) in enumerate(zip(sentences, predictions)):

        total_uri.append(row['URI'])
        total_identifier.append(row['Identifier'])
        total_location.append(row['Location'])
        total_time.append(row['Time'])
        total_event.append(row['Event'])
        total_sentence_num.append(i)
        total_sentences.append(sentence)
        total_subject.append(prediction[0])
        total_relation.append(prediction[1])
        total_object.append(prediction[2])


new_data = pd.DataFrame({'URI': total_uri, 'Identifier': total_identifier, 'Location': total_location, 'Time': total_time, 'Sentence_num': total_sentence_num, 'Sentence': total_sentences, 'Subject': total_subject, 'Relation': total_relation, 'Object': total_object, 'Event': total_event})

new_data = new_data[~((new_data['Sentence'].str.len() < 25) & new_data['Sentence'].str.contains('/'))].reset_index(drop=True)

new_data.to_csv('final_data_with_predictions.csv')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mike-\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/1122 [00:00<?, ?it/s]C:\Users\mike-\Documents\VU\Eurecom\KG_mapping\venv\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 1122/1122 [2:42:08<00:00,  8.67s/it] 


# Merge same entities
### Entity coreference resolution

In [2]:
# This code will load the clusters from a text file, and merge the nodes in the cluster together
import ast
from resources import node_creation
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace

data = pd.read_csv('Data/final_data_with_predictions_new.csv') #Load the original dataset
with open('Data/cluster_data/event_clusters_0.85.txt') as f: #Open the generated cluster file
    cluster_doc = f.readlines()

In [5]:
graph = Graph()
graph.parse('Data/examples/event_article_graph_complete_new.ttl')

for cluster in cluster_doc:

    if cluster.startswith('['):
        cluster = ast.literal_eval(cluster)

        prev_mention_uri = None
        if len(cluster) != 1:
            for i, mention in enumerate(cluster):
                mention = mention.split('_')

                sentence = data[(data['URI'] == mention[1]) & (data['Sentence_num'] == int(mention[2]))]['Sentence'].values[0]
                sentence_uri = node_creation('', sentence, base_add='/sentence') #Generate the URI for the sentence

                mention_uri = node_creation('', mention[0] + str(sentence_uri), base_add='/subject')

                #determine subject or object
                if data[(data['URI'] == mention[1]) & (data['Sentence_num'] == int(mention[2]))]['Subject'].values[0] == mention[0]:
                    mention_uri = node_creation('', mention[0] + str(sentence_uri), base_add='/subject')

                else:
                    mention_uri = node_creation('', mention[0] + str(sentence_uri), base_add='/object')

                #print(mention_uri)

                if i != 0:
                    graph.add((prev_mention_uri, owl.sameAs, mention_uri))


                prev_mention_uri = mention_uri



http://kflow.eurecom.fr/subject/35242267-f003-5a05-9c4e-02eb1bd1bd73
http://kflow.eurecom.fr/subject/82cb0354-3a51-53f9-a555-23ab0769be42
http://kflow.eurecom.fr/subject/82cb0354-3a51-53f9-a555-23ab0769be42
http://kflow.eurecom.fr/object/14fa475d-55b1-5a8e-aca5-4fdaad1c767f
http://kflow.eurecom.fr/subject/00b9e053-5954-527e-b3ae-1f09a5a22b7a
http://kflow.eurecom.fr/subject/f19f0f34-f6f9-50bb-84e3-c283cb7d8016
http://kflow.eurecom.fr/subject/e14761c0-363e-5ee8-bd11-c57bdf88bae2
http://kflow.eurecom.fr/subject/35242267-f003-5a05-9c4e-02eb1bd1bd73
http://kflow.eurecom.fr/subject/50fc9e7d-0fcc-5ae8-ae2a-444b5d682ae3
http://kflow.eurecom.fr/subject/35242267-f003-5a05-9c4e-02eb1bd1bd73
http://kflow.eurecom.fr/subject/50fc9e7d-0fcc-5ae8-ae2a-444b5d682ae3
http://kflow.eurecom.fr/subject/50fc9e7d-0fcc-5ae8-ae2a-444b5d682ae3
http://kflow.eurecom.fr/subject/50fc9e7d-0fcc-5ae8-ae2a-444b5d682ae3
http://kflow.eurecom.fr/object/c968139c-a2e5-5d8b-b895-f60510300dfd
http://kflow.eurecom.fr/object/a9672

# Select most relevant information from the graph

In [16]:
#Get some statistics first

g = Graph()
g.parse("Data/examples/event_article_graph_complete.ttl")

query = """
PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
SELECT ?URI ?
WHERE {
    ?s a sem:Event;
        ?p ?o
}"""

qres = g.query(query)
for row in qres:
    print(f"num_pred: {row.num_predicates} - num_events: {row.num_events}: on average: {int(row.num_predicates) / int(row.num_events)}")

query = """
PREFIX sem: <http://semanticweb.cs.vu.nl/2009/11/sem/>
PREFIX rnews: <http://iptc.org/std/rNews/2011-10-07#>
SELECT (COUNT(?p) as ?num_predicates) (COUNT(DISTINCT ?s) as ?num_articles)
WHERE {
    ?s a rnews:Article;
        ?p ?o
}"""

qres = g.query(query)
for row in qres:
    print(f"num_pred: {row.num_predicates} - num_article: {row.num_articles}: on average: {int(row.num_predicates) / int(row.num_articles)}")

504 - 86: on average: 5.8604651162790695
25852 - 1122: on average: 23.040998217468804
